In [1]:
import os
import torch
import numpy as np
from PIL import Image
import open_clip

import requests
import pickle
from io import BytesIO
from transformers import AutoModel, AutoImageProcessor

import faiss
import json

import cv2
import functions

KeyboardInterrupt: 

In [ ]:
# import list of collections and lists of urls
import pickle

with open('./data/scrap/cards_metadata.pickle', 'rb') as f:
    cards_metadata = pickle.load(f)

with open('./data/scrap/cards_collections.pickle', 'rb') as f:
    cards_collections = pickle.load(f)

with open('./data/scrap/cards_urls.pickle', 'rb') as f:
    cards_urls = pickle.load(f)   
    
cards_collections = sorted(cards_collections, key=str.lower)

In [ ]:
model, processor, device =  functions.load_embbedings_model()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# creates vector index:
index = faiss.IndexFlatL2(768)

In [ ]:
cards_collections[:4]

['AOR', 'ASR', 'base1', 'base2']

In [ ]:
embeddings = []
schema = []
sub_cards_collections = cards_collections

for idx, card_collection in enumerate(sub_cards_collections):
    embeddings = []
    metadata = []
    temp_metadata = [p for p in cards_metadata if p['collection'] == card_collection]
    for card in temp_metadata:
        try :
            emb = functions.get_embedding(card['url'],model, processor, device)
            embeddings.append(emb)
            metadata.append({"id": card_collection + "_" + card['name'], "collection": card_collection}) 
        except : 
            print("ERROR - Card : ", card['name'], " could not be loaded")
    # Ajout à la collection faiss
    index.add(np.vstack(embeddings).astype("float32"))
    schema.extend(metadata)
    print(str(idx+1),"/",str(len(sub_cards_collections)), " - ", card_collection, " has been added to chromadb")
    
    #save data
    faiss.write_index(index, f"./faiss_db/pokemon-cards.index")
    with open(f"./faiss_db/pokemon-cards_schema.json", "w", encoding="utf-8") as f:
        json.dump(schema, f, indent=2, ensure_ascii=False)

/opt/miniconda3/envs/pokemon-faiss/lib/python3.11/site-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1 / 147  -  AOR  has been added to chromadb
2 / 147  -  ASR  has been added to chromadb
3 / 147  -  base1  has been added to chromadb
4 / 147  -  base2  has been added to chromadb
5 / 147  -  base3  has been added to chromadb
6 / 147  -  base4  has been added to chromadb
7 / 147  -  base5  has been added to chromadb
8 / 147  -  base6  has been added to chromadb
9 / 147  -  basep  has been added to chromadb
10 / 147  -  BCR  has been added to chromadb
11 / 147  -  BKP  has been added to chromadb
12 / 147  -  BKT  has been added to chromadb
13 / 147  -  BLK  has been added to chromadb
14 / 147  -  BLW  has been added to chromadb
15 / 147  -  bp  has been added to chromadb
16 / 147  -  BRS  has been added to chromadb
17 / 147  -  BST  has been added to chromadb
ERROR - Card :  BUS_112a_R_EN_LG.png  could not be loaded
ERROR - Card :  BUS_115a_R_EN_LG.png  could not be loaded
18 / 147  -  BUS  has been added to chromadb
ERROR - Card :  BWP_097_R_EN_LG.png  could not be loaded
19 / 147  -  

In [ ]:
index.ntotal

19505